In [1]:
from helper.setup import *
from helper.pycuda_setup import *

#%run -i make_matrices.py

states = pd.read_csv('./data/states.csv',index_col=0)
players = pd.read_json('./data/players.json',orient='index')
rbi = pd.read_csv('./data/rbi.csv',index_col=0)
valid = rbi >= 0
rbi[~valid] = 0
rbi = rbi.values

outs = states['outs']
T = np.stack(players['T']).astype('float32')

def simulate_game(sim, print_level=0):
    div_len = 95
    inning = 0
    total_ABs = 0
    AB_idx = 0
    state_in = 0
    state_out = 0
    score = 0
    batter_idx = lineup[AB_idx];

    for inning in range(1,10):
        state_in = 0;

        if(print_level >= 2):
            print("*"*div_len)
            print("Sim %d, START OF INNING %d, runs %d, outs %d, AB %d, bat %d, s_in %d"
                  %(sim, inning, score, outs[state_in], AB_idx, batter_idx, state_in))
            if(print_level >= 3):
                print()

        while(outs[state_in] < 3):
            state_out = 0            
            r = np.random.rand()

            if(print_level >= 3):
                print("Sim %d, inn %d, runs %d, outs %d, AB %d, bat %d, s_in %d, r = %f"
                      %(sim, inning, score, outs[state_in], AB_idx, batter_idx, state_in, r))

            while(r > 0):
                prob = T[batter_idx, state_in, state_out]
                r -= prob
                if(print_level >= 4):
                    print("Sim %d, inn %d, runs %d, outs %d, AB %d, bat %d, s_out %d, prob=%f, r_left=%f"
                          %(sim, inning, score, outs[state_in], AB_idx, batter_idx, state_out, prob, r))
                state_out += 1
            state_out -= 1
            score += rbi[state_in, state_out]
            state_in = state_out
            
            if(print_level >= 3):
                print("Sim %d, inn %d, runs %d, outs %d, AB %d, bat %d, s_out %d, r_left = %f\n"
                      %(sim, inning, score, outs[state_out], AB_idx, batter_idx, state_out, r))

            total_ABs += 1
            AB_idx += 1
            AB_idx %= 9
            batter_idx = lineup[AB_idx]
            
        
        if(print_level >= 2):
            print("Sim %d, END OF INNING %d, runs %d, outs %d, AB %d, bat %d, s_out %d"
                  %(sim, inning, score, outs[state_out], AB_idx, batter_idx, state_out))
            print('*'*div_len)

    if(print_level >= 1):
            print('#'*div_len)
            print("Sim %d, END OF GAME, inning %d, FINAL SCORE %d, outs %d, AB %d, bat %d, state_out %d, total ABs %d"
                  %(sim, inning, score, outs[state_out], AB_idx, batter_idx, state_out, total_ABs))            
            print('#'*div_len)        

    return score

#Randomly selects 9 batters
num_players = players.shape[0]
lineup = np.random.permutation(np.arange(num_players))[:9]


def get_summary_stats(v):
    ss = pd.Series(v.ravel()).describe()
    ss['SE'] = ss['std'] / np.sqrt(ss['count'])
    return ss

In [2]:
%%time
print_level = 0
num_sims = 1024*8
reps = 10

for rep in range(reps):
    scores = np.zeros(num_sims,dtype='uint16')
    for i in range(num_sims):
        scores[i] = simulate_game(i, print_level)

    ss = get_summary_stats(scores)
    if rep == 0:
        sum_stats = pd.DataFrame(columns=ss.index)
    sum_stats.loc[rep] = ss

mu = sum_stats['mean'].mean()
sum_stats['In 95% CI?'] = (sum_stats['mean'] - mu).abs() < 1.96*sum_stats['SE']
sum_stats.ix['AVERAGE'] = sum_stats.mean(axis=0)
#roll average row to top
sum_stats = pd.concat([sum_stats.ix[['AVERAGE']],sum_stats.ix[0:-1]])
display(sum_stats)

,count,mean,std,min,25%,50%,75%,max,SE,In 95% CI?
AVERAGE,8192.000,8.297,3.539,0.000,6.000,8.000,10.500,25.700,0.039,1.000
0,8192.000,8.349,3.501,0.000,6.000,8.000,11.000,25.000,0.039,1.000
1,8192.000,8.304,3.542,0.000,6.000,8.000,11.000,26.000,0.039,1.000
2,8192.000,8.268,3.504,0.000,6.000,8.000,10.000,25.000,0.039,1.000
3,8192.000,8.328,3.527,0.000,6.000,8.000,11.000,26.000,0.039,1.000
4,8192.000,8.287,3.518,0.000,6.000,8.000,10.000,25.000,0.039,1.000
5,8192.000,8.339,3.573,0.000,6.000,8.000,11.000,24.000,0.039,1.000
6,8192.000,8.224,3.546,0.000,6.000,8.000,10.000,27.000,0.039,1.000
7,8192.000,8.320,3.550,0.000,6.000,8.000,11.000,26.000,0.039,1.000
8,8192.000,8.297,3.569,0.000,6.000,8.000,10.000,26.000,0.039,1.000


CPU times: user 2min 2s, sys: 16 ms, total: 2min 3s
Wall time: 2min 3s
